In [1]:
import numpy as np
import pickle 
from gensim.models import Word2Vec
import gensim

In [2]:
txt = open('data/nyt/vec.txt', 'r')

In [3]:
txt_lines = [line for line in txt.readlines()]

In [4]:
txt_count = int(txt_lines[0].split()[0])
vec_dim = int(txt_lines[0].split()[1])

In [5]:
word_vec_map = {}
for line in txt_lines[1:]:
    temp_list = line.split()
    word = temp_list[0]
    vec = []
    for number in temp_list[1:]:
        vec.append(float(number))
    word_vec_map[word] = vec

In [9]:
nyt_relation = {}
inverse_nyt_rela = {}
with open("data/nyt/relation2id.txt", 'r') as f:
    for line in f.readlines():
        line = line.strip('\n')
        if line[0] == '/':
            items = str.split(line, ' ')
            nyt_relation[items[1]] = items[0]
            inverse_nyt_rela[items[0]] = items[1]
        else:
            items = str.split(line, ' ')
            nyt_relation[items[1]] = items[0]
            inverse_nyt_rela[items[0]] = items[1]

In [65]:
nyt_relation

{'0': 'NA',
 '1': '/location/neighborhood/neighborhood_of',
 '10': '/location/it_region/capital',
 '11': '/people/family/members',
 '12': '/location/us_state/capital',
 '13': '/location/us_county/county_seat',
 '14': '/people/profession/people_with_this_profession',
 '15': '/location/br_state/capital',
 '16': '/location/in_state/legislative_capital',
 '17': '/sports/sports_team/location',
 '18': '/people/person/religion',
 '19': '/location/in_state/judicial_capital',
 '2': '/location/fr_region/capital',
 '20': '/business/company_advisor/companies_advised',
 '21': '/people/family/country',
 '22': '/time/event/locations',
 '23': '/business/company/place_founded',
 '24': '/location/administrative_division/country',
 '25': '/people/ethnicity/included_in_group',
 '26': '/location/mx_state/capital',
 '27': '/location/province/capital',
 '28': '/people/person/nationality',
 '29': '/business/person/company',
 '3': '/location/cn_province/capital',
 '30': '/business/shopping_center_owner/shoppin

In [66]:
inverse_nyt_rela

{'/base/locations/countries/states_provinces_within': '5',
 '/broadcast/content/location': '39',
 '/broadcast/producer/location': '38',
 '/business/business_location/parent_company': '40',
 '/business/company/advisors': '31',
 '/business/company/founders': '6',
 '/business/company/locations': '46',
 '/business/company/major_shareholders': '37',
 '/business/company/place_founded': '23',
 '/business/company_advisor/companies_advised': '20',
 '/business/person/company': '29',
 '/business/shopping_center/owner': '32',
 '/business/shopping_center_owner/shopping_centers_owned': '30',
 '/film/film/featured_film_locations': '42',
 '/film/film_festival/location': '52',
 '/film/film_location/featured_in_films': '51',
 '/location/administrative_division/country': '24',
 '/location/br_state/capital': '15',
 '/location/cn_province/capital': '3',
 '/location/country/administrative_divisions': '49',
 '/location/country/capital': '47',
 '/location/country/languages_spoken': '7',
 '/location/de_state/c

In [12]:
common_relation = pickle.load(open('data/common_relations.pkl', 'rb'))

In [88]:
common_relation

{'NA': [3, 'P0', ['0']],
 'capital': [2,
  'P36',
  ['2', '3', '10', '12', '15', '26', '27', '41', '44', '47']],
 'country': [6, 'P17', ['21', '24']],
 'founder': [7, 'P112', ['6']],
 'location': [8, 'P276', ['17', '22', '38', '39', '46', '52']],
 'place of birth': [0, 'P19', ['8']],
 'place of burial': [1, 'P119', ['34']],
 'place of death': [4, 'P20', ['9']],
 'religion': [5, 'P140', ['18']]}

In [6]:
class MySentences(object):
    def __init__(self, files):
        self.files = files
 
    def __iter__(self):
#         for fname in os.listdir(self.dirname):
#             for line in open(os.path.join(self.dirname, fname)):
#                 yield line.split()
        for filename in self.files:
            for line in open(filename):
                items = line.strip('\n').split('\t')
                words = items[-1][:-9].split()
                words = [word.strip('.') for word in words if len(word) > 0]
                yield words

def GetEntityPos(entity1, entity2, sentence):
    pos_1 = 0
    pos_2 = 0
    for i in range(len(sentence)):
        if sentence[i] == entity1:
            pos_1 = i
        if sentence[i] == entity2:
            pos_2 = i
    pos_first = min(pos_1, pos_2)
    pos_second = pos_1 + pos_2 - pos_first
    return [pos_first, pos_second]

def GetEmbeddingData(entity_1, entity_2, words, max_length, vec_size, pos_size, word_2_vec_model):
    word_embedding = np.zeros((max_length, vec_size))
    pos_embedding = np.zeros((max_length, pos_size))
    (entity_pos_1, entity_pos_2) = GetEntityPos(entity_1, entity_2, words)
    pos_embedding[entity_pos_1] = np.ones(pos_size)
    pos_embedding[entity_pos_2] = np.ones(pos_size)
    for i in range(len(words)):
        try:
            word_embedding[i] = word_2_vec_model[words[i]]
        except:
            print(words)
    input_embedding = np.concatenate((word_embedding, pos_embedding), axis = 1)
    input_embedding = input_embedding.reshape(max_length, -1)
    
    return input_embedding

In [7]:
is_training = True
txts = ['data/nyt/train.txt', 'data/nyt/test.txt']
if is_training:
    
    # temp_file = open(wiki_file, 'r')
    # temp_file_lines = [line for line in temp_file.readlines()]
    sentences = MySentences(txts)
    model = gensim.models.Word2Vec(sentences, min_count=1, size = 50)
    model.save('Preprocess_model_pkl/nyt_vec_model.pkl')
else:
    model = gensim.models.Word2Vec.load('Preprocess_model_pkl/nyt_vec_model.pkl')

In [133]:
train_txt = open('data/nyt/train.txt', 'r')
train_lines = [line for line in train_txt]

In [134]:
for line in train_lines:
    count += 1
print(count)

570088


In [151]:
count = 0
relation_labels = []
max_length = 100
vec_size = 50
pos_size = 5
train_data = np.zeros((442506, max_length, pos_size+vec_size))
for line in train_lines:
    flag = 0
    items = line.strip('\n').split('\t')
    if (count%10000) == 0:
        print("count is: ", count)
    if items[4] in inverse_nyt_rela:
        relation_name = inverse_nyt_rela[items[4]]
    else:
        continue
    for (key, relation) in common_relation.items():
        nyt_list = relation[2]
        for nyt_rela in nyt_list:
            if relation_name == (nyt_rela):
                relation_id = relation[0]
                flag = 1
                relation_labels.append(relation_id)
                break
    if flag == 0:
        continue
    entity1 = items[2]
    entity2 = items[3]
    words = items[-1][:-9].split()
    words = [word.strip('.') for word in words if len(word) > 0]
    if len(words) > max_length:
        continue
    input_embedding = GetEmbeddingData(entity1, entity2, words, max_length, vec_size, pos_size, model)
    train_data[count] = input_embedding
    count += 1

#     if train_data is None:
#         train_data = input_embedding
#     else:
#         train_data = np.concatenate((train_data, input_embedding),axis = 0)
print(count)

count is:  0
count is:  0
count is:  0
count is:  0
count is:  0
count is:  0
count is:  0
count is:  0
count is:  0
count is:  0
count is:  0
count is:  0


/Users/sun/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


count is:  10000
count is:  20000
count is:  30000
count is:  40000
count is:  50000
count is:  60000
count is:  70000
count is:  80000
count is:  90000
count is:  100000
count is:  110000
count is:  120000
count is:  130000
count is:  140000
count is:  150000
count is:  160000
count is:  170000
count is:  180000
count is:  190000
count is:  200000
count is:  210000
count is:  220000
count is:  230000
count is:  240000
count is:  250000
count is:  260000
count is:  270000
count is:  280000
count is:  290000
count is:  300000
count is:  310000
count is:  320000
count is:  330000
count is:  340000
count is:  350000
count is:  360000
count is:  370000
count is:  380000
count is:  390000
count is:  400000
count is:  410000
count is:  420000
count is:  430000
439693


In [156]:
relation_label_np = np.array(relation_labels)

In [157]:
np.save('data/processed_data/nyt_train_label.np', relation_label_np)

In [158]:
np.save('data/processed_data/nyt_train_data.np', train_data)

In [13]:
test_txt = open('data/nyt/test.txt', 'r')
test_lines = [line for line in test_txt]
count = 0
test_relation_labels = []
max_length = 100
vec_size = 50
pos_size = 5
test_data = np.zeros((442506, max_length, pos_size+vec_size))
for line in test_lines:
    flag = 0
    items = line.strip('\n').split('\t')
    if (count%10000) == 0:
        print("count is: ", count)
    if items[4] in inverse_nyt_rela:
        relation_name = inverse_nyt_rela[items[4]]
    else:
        continue
    for (key, relation) in common_relation.items():
        nyt_list = relation[2]
        for nyt_rela in nyt_list:
            if relation_name == (nyt_rela):
                relation_id = relation[0]
                flag = 1
                test_relation_labels.append(relation_id)
                break
    if flag == 0:
        continue
    entity1 = items[2]
    entity2 = items[3]
    words = items[-1][:-9].split()
    words = [word.strip('.') for word in words if len(word) > 0]
    if len(words) > max_length:
        continue
    input_embedding = GetEmbeddingData(entity1, entity2, words, max_length, vec_size, pos_size, model)
    test_data[count] = input_embedding
    count += 1

#     if train_data is None:
#         train_data = input_embedding
#     else:
#         train_data = np.concatenate((train_data, input_embedding),axis = 0)
print(count)
test_data = test_data[0:count-1]
test_labels_np = np.array(test_relation_labels)
np.save('data/processed_data/nyt_test_label.np', test_labels_np)
np.save('data/processed_data/nyt_test_data.np', test_data)

count is:  0
count is:  10000
count is:  20000
count is:  30000
count is:  40000
count is:  50000
count is:  60000
count is:  70000
count is:  80000
count is:  90000
count is:  100000
count is:  110000
count is:  120000
count is:  130000
count is:  140000
count is:  150000
count is:  160000
167387


In [98]:
inverse_nyt_rela

{'/base/locations/countries/states_provinces_within': '5',
 '/broadcast/content/location': '39',
 '/broadcast/producer/location': '38',
 '/business/business_location/parent_company': '40',
 '/business/company/advisors': '31',
 '/business/company/founders': '6',
 '/business/company/locations': '46',
 '/business/company/major_shareholders': '37',
 '/business/company/place_founded': '23',
 '/business/company_advisor/companies_advised': '20',
 '/business/person/company': '29',
 '/business/shopping_center/owner': '32',
 '/business/shopping_center_owner/shopping_centers_owned': '30',
 '/film/film/featured_film_locations': '42',
 '/film/film_festival/location': '52',
 '/film/film_location/featured_in_films': '51',
 '/location/administrative_division/country': '24',
 '/location/br_state/capital': '15',
 '/location/cn_province/capital': '3',
 '/location/country/administrative_divisions': '49',
 '/location/country/capital': '47',
 '/location/country/languages_spoken': '7',
 '/location/de_state/c

In [10]:
inverse_nyt_rela['/people/ethnicity/includes_groups'] = '25'
inverse_nyt_rela['/sports/sports_team_location/teams'] = '17'

In [67]:
test = train_lines[0].strip('\n').split('\t')[-1][:-9].split()